## Import

In [73]:
import os
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import urllib
import json

# Import csv file

In [74]:
df = pd.read_csv("C:\\Users\\odeya\\Desktop\\TaskPython\\destinations_LP_crawler_Ex5 (1).csv", encoding='latin-1')
df.head()

,city,city_LP
0,Antalya,mediterranean-coast/antalya
1,Athens,Athens
2,Ayia-napa,the-republic-of-cyprus/Agia-napa
3,Bangkok,Bangkok
4,Barcelona,Barcelona


# Get country


In [75]:
def get_country(city):
    api_key = 'api_key'
    try:
        url = "https://maps.googleapis.com/maps/api/geocode/json?address=%s&key=%s" % (city,api_key)
        response = requests.get(url)
        if response.status_code != 200:
            return 'Error - API request didnt work'
        else:
            country = response.json()['results'][0]['formatted_address'].split(",")[-1]
            return country
    except:
        return 'Error in get_country func'

# Get description of the city

In [76]:
def get_description(df):
    desc = []
    i = 0
    for i in range(len(df)):
        try:
            link = 'https://www.lonelyplanet.com/' + (df["country"][i]).strip() + "/" + df["city_LP"][i]
            response = requests.get(link)
            if response.status_code != 200:
                desc.append('Error - scrapping didnt work')
            else:
                soup = BeautifulSoup(response.content, 'html.parser')
                description = soup.find("div",class_="readMore_content___5EuR relative overflow-hidden max-h-96 is-max wysiwyg").get_text()
                desc.append(description)
        except:
            desc.append('Error in get_description func')
    return desc

# Main

In [72]:
df['country'] = (df['city'].apply(get_country))

df.loc[22,"country"]="poland"
df.loc[37,"country"]="ukraine"
df.loc[56,"country"]="ukraine"

df['Description'] = get_description(df)

df_final = pd.DataFrame()
df_final['City'] = df['city']
df_final['Country'] = df['country']
df_final['Description'] = df['Description']

print(df_final)
df_final.to_csv("C:/Users/odeya/Desktop/TaskPython/LP_destinations.csv", index = False, header=True)

         City       Country                                        Description
0     Antalya        Turkey  Once seen simply as the gateway to the Turkish...
1      Athens        Greece  With equal measures of grunge and grace, Athen...
2   Ayia-napa        Cyprus  Endless blue-sky days and a coastline riddled ...
3     Bangkok      Thailand  Same same, but different. This Thailish T-shir...
4   Barcelona         Spain  Barcelona is an enchanting seaside city with b...
..        ...           ...                                                ...
71     Venice         Italy  Imagine the audacity of building a city of mar...
72     Vienna       Austria  Baroque streetscapes and imperial palaces set ...
73     Warsaw        Poland  A phoenix arisen from the ashes, Poland's capi...
74  Zakynthos        Greece  Zakynthos, also known by its Italian name Zant...
75     Zurich   Switzerland  Culturally vibrant, efficiently run and attrac...

[76 rows x 3 columns]
